# Black–Scholes–Merton Pricing Formula

In [4]:
from numpy import pi, exp, log
from scipy.stats import norm

### Price of a European Call and Put Option

The standard normal probability density function

In [5]:
def norm_pdf(x):
    return (1/((2*pi)**0.5))*exp(-0.5*x*x)

Next, we need to define:

$d_j = \frac{\log(\frac{S}{K})+(r+(-1)^{j-1} \frac{\sigma^2T}{2})}{\sigma \sqrt{T}}$

In [6]:
def d_j(j, S, K, r, v, T):
    return (log(S/K) + (r + ((-1)**(j-1))*0.5*v*v)*T)/(v*(T**0.5))

Price of a European call option struck at K, with spot S, constant rate r,
constant vol v (over the life of the option) and time to maturity T:

In [7]:
def call_price(S, K, r, v, T):
    return S*norm.cdf(d_j(1, S, K, r, v, T))-K*exp(-r*T) * norm.cdf(d_j(2, S, K, r, v, T))

Price of a European put option struck at K, with spot S, constant rate r, constant vol v 
(over the life of the option) and time to maturity T:

In [9]:
def put_price(S, K, r, v, T):
    return -S*norm.cdf(-d_j(1, S, K, r, v, T))+K*exp(-r*T) * norm.cdf(-d_j(2, S, K, r, v, T))

put-call parity wih risk free interest rate:

In [31]:
def parity(S, K, r, T):
    return S-K*exp(-r*T)

### Examples

Calculate both the price of a call and a put for: $S_0$ = K = 100, time to expiry 1 month, i.e., $\tau$ = 30/252, r = 5%, and $\sigma$ = 20%

In [22]:
put = put_price(100, 100, 0.05, 0.2, 30/252)
call = call_price(100, 100, 0.05, 0.2, 30/252)
print("The price of Put option is %s" %(round(put, 2)))
print("The price of Call option is %s" %(round(call, 2)))

The price of Put option is 2.46
The price of Call option is 3.05


Check that the Put-Call parity relation holds in the above example.

In [34]:
eps = 1e-7
par = parity(100, 100, 0.05,30)
if ((call-put)-par) < eps:
    print('Put-Call parity holds.')
else:
    print('Put-Call parity does not hold.')

Put-Call parity holds.


## Implied Volatility

The implied volatility is, by definition, the value of $\sigma$ for which the function $$f(\sigma)= C_{BSM}(S_0,K,T,r;\sigma)−C_M(K,T)$$
is zero. Here, $C_M (K, T)$ and $C_{BSM} (S_0, K, T ; \sigma)$ are the market price of an European Call Option and the Black–Scholes–Merton price, respectively.

Assuming we have option price data (downloadable from Bloomerg terminal or Yahoo! Finance), we can calculate the implied volatiltiy of an option using tools such as the bisection or the Secant method.

### Bisection method

In [39]:
def bisection(f, a, b, TOL=0.0001):    
    c = (a+b)/2.0
    while abs(b-a)/2>TOL:
        if f(c)==0:
            return (c)
        elif f(c)*f(a) < 0:
            b = c
        else: a = c
        c = (a+b) / 2.0
    return (c)      

### Secant method

In [40]:
def secant(f,a,b,TOL=0.0001):
    for i in range(1000):    
        if f(b)-f(a) < TOL:
            return b
        x_temp = b - (f(b)*(b-a)*1.0)/(f(b)-f(a))
        a = b
        b = x_temp
    return b

We also need to obtain the spot price of the stock and the market risk-free interest rate, and define the function. An example is provided below

In [41]:
S_Amzn=840.35
r_fed=0.0051
def f(v):
    return call_price(S_Amzn,K_Amzn,r_fed,v,time)-price

You can load the option prices and save the implied volatilities as a pandas dataframe. If you obtain the data for stocks with different strike prices, you can plot the volatility smile.

## Greeks

Calculating the derivatives using the BSM formula is straightforward. Here we calculate the derivative of the call option price with respect to S (Delta), and $\sigma$ (Vega) and the second derivative with respect to S (Gamma)

In [45]:
def delta(S,K,r,v,T):
    return norm.cdf(d_j(1, S, K, r, v, T))


def vega(S,K,r,v,T):
    return S*norm_pdf(d_j(1, S, K, r, v, T))*sqrt(T)


def gamma(S,K,r,v,T):
    return norm_pdf(d_j(1, S, K, r, v, T))/(S*v*sqrt(T))